In [ ]:
!pip install kaggle-environments

In [ ]:
import random
from kaggle_environments import make, evaluate
from tqdm import tqdm

In [ ]:
def always_rock(observation, configuration):
    return 0
def always_scissors(observation, configuration):
    return 1
def always_paper(observation, configuration):
    return 2



In [ ]:
def random_agent(observation, configuration):
    return random.randint(0, 2)


In [ ]:
def counter_previous(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    opponent_last_move = observation.lastOpponentAction
    return (opponent_last_move + 1) % 3
def win_previous(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    opponent_last_move = observation.lastOpponentAction
    return (opponent_last_move + 2) % 3
def lose_previous(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    opponent_last_move = observation.lastOpponentAction
    return (opponent_last_move + 1) % 3
def last_mirror(observation, configuration):
    if observation.step == 0:
        return 0  # Начинаем с камня
    return observation.lastOpponentAction


In [ ]:
import random

def weighted_random_agent(observation, configuration):
    weights = [0.4, 0.3, 0.3]  # Веса для камня, ножниц и бумаги соответственно
    return random.choices([0, 1, 2], weights=weights, k=1)[0]
def cyclic_agent(observation, configuration):
    return observation.step % 3



In [106]:
%%writefile counter_most_common_agent.py
opponent_moves = []
def counter_most_common_agent(observation, configuration):
    global opponent_moves
    if observation.step == 0:
        return 0  # Начинаем с камня
    else:
        opponent_moves.append(int(observation.lastOpponentAction))
    move_counts = [opponent_moves.count(0), opponent_moves.count(1), opponent_moves.count(2)]
    most_common_move = move_counts.index(max(move_counts))
    return (most_common_move + 1) % 3


Writing counter_most_common_agent.py


In [101]:
%%writefile analyze_sequence.py
import random
opponent_history = []
def analyze_sequence(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    
    global opponent_history
    if observation.step > 0:
        opponent_history.append(int(observation.lastOpponentAction))
    
    if len(opponent_history) < 3:
        return random.randint(0, 2)
    
    # Найдем последнюю последовательность ходов противника
    last_sequence = tuple(opponent_history[-3:])
    
    # Создаем словарь для хранения частот следующих ходов
    next_move_freq = {0: 0, 1: 0, 2: 0}
    
    for i in range(len(opponent_history) - 3):
        current_sequence = tuple(opponent_history[i:i+3])
        if current_sequence == last_sequence:
            next_move = opponent_history[i+3]
            next_move_freq[next_move] += 1
    
    # Выбираем наиболее частый следующий ход
    most_frequent_move = max(next_move_freq, key=next_move_freq.get)
    
    # Возвращаем ход, который победит наиболее частый следующий ход
    return (most_frequent_move + 1) % 3


Writing analyze_sequence.py


In [100]:
%%writefile probabilistic_predictor.py
import random

opponent_history = []

def probabilistic_predictor(observation, configuration):
    global opponent_history
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        opponent_history.append(int(observation.lastOpponentAction))
    # Подсчитываем частоту каждого хода противника
    move_counts = [0, 0, 0]
    for move in opponent_history:
        move_counts[move] += 1
    
    # Вычисляем вероятности каждого хода
    total_moves = sum(move_counts)
    probabilities = [count / total_moves for count in move_counts]
    
    # Выбираем ход, который победит наиболее вероятный ход противника
    most_probable_move = probabilities.index(max(probabilities))
    return (most_probable_move + 1) % 3


Overwriting probabilistic_predictor.py


In [123]:
agents = [
    always_rock,
    always_scissors,
    always_paper,
    random_agent,
    counter_previous,
    win_previous,
    lose_previous,
    last_mirror,
    weighted_random_agent,
    cyclic_agent,
    'analyze_sequence.py',
    'counter_most_common_agent.py',
    'probabilistic_predictor.py'
]
agents_names = [   'always_rock',
    'always_scissors',
    'always_paper',
    'random_agent',
    'counter_previous',
    'win_previous',
    'lose_previous',
    'last_mirror',
    'weighted_random_agent',
    'cyclic_agent',
    'analyze_sequence.py',
    'counter_most_common_agent.py',
    'probabilistic_predictor.py']

# Функция для проведения турнира
def tournament(agents, num_games=10):
    results = {agent: 0 for agent in agents_names}
    for i in tqdm(range(len(agents))):
        for j in range(i + 1, len(agents)):
            agent1 = agents[i]
            agent2 = agents[j]
            scores = evaluate("rps", [agent1, agent2], num_episodes=num_games)
            results[agents_names[i]] += sum([score[0] for score in scores])
            results[agents_names[j]] += sum([score[1] for score in scores])
    return results

# Запускаем турнир
results = tournament(agents)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [34:27<00:00, 159.00s/it]


In [124]:
for k,v in zip(results.keys(),results.values()) :
    print(k, v)

always_rock -40028.0
always_scissors -39015.0
always_paper -40803.0
random_agent 518.0
counter_previous 27131.0
win_previous -39769.0
lose_previous 29743.0
last_mirror -17435.0
weighted_random_agent -2769.0
cyclic_agent -10091.0
analyze_sequence.py 82640.0
counter_most_common_agent.py 28311.0
probabilistic_predictor.py 21567.0
